In [3]:
import os
import keras as K
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import LeakyReLU
from keras.utils import np_utils
from keras.optimizers import SGD, RMSprop, Adam
from keras.wrappers.scikit_learn import KerasClassifier
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from keras import regularizers

In [4]:
file = 'dataset_1'

train_datagen = K.preprocessing.image.ImageDataGenerator()

validation_datagen = K.preprocessing.image.ImageDataGenerator()

test_datagen = K.preprocessing.image.ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
        os.path.join(file, 'train'),
        target_size=(50, 50),
        batch_size=1,
        color_mode='grayscale',
        class_mode='categorical',
        )

validation_generator = validation_datagen.flow_from_directory(
        os.path.join(file, 'validation'),
        target_size=(50, 50),
        batch_size=1,
        color_mode='grayscale',
        class_mode="categorical",
        )

test_generator = validation_datagen.flow_from_directory(
        os.path.join(file, 'test'),
        target_size=(50, 50),
        batch_size=1,
        color_mode='grayscale',
        class_mode="categorical",
        )

Found 12556 images belonging to 28 classes.
Found 1710 images belonging to 28 classes.
Found 3515 images belonging to 28 classes.


In [5]:
def runGenerator(generator):
    X = []
    y = []
    for i in range(len(generator)):
        X_ex, y_ex = generator.next()
        X.append(X_ex)
        y.append(y_ex)
    return X, y

def F1ScoreCalculator(test_generatorm, model):
    X_test = []
    y_test = []
    y_pred = []
    for i in range(len(test_generator)):
        X_ex, y_ex = test_generator.next()
        X_test.append(X_ex)
        y_test.append(y_ex.argmax(axis=-1))
        y_pred.append(model.predict(X_ex).argmax(axis=-1))

    return f1_score(y_test, y_pred, average='micro')

In [6]:
def Create_model(kernel_sizes=5, neurons_bk1=20, neurons_bk2=50, learn_rate=0.001, optomizer='sgd', activation='relu'):
    print('kernel_sizes= {} | neurons_bk1= {} | neurons_bk2= {} | learn_rate={} | optomizer={} | activation={}'.format(
        kernel_sizes, 
        neurons_bk1, 
        neurons_bk2, 
        learn_rate,
        optomizer,
        activation))
    
    model = Sequential()

    model.add(Conv2D(neurons_bk1, kernel_size=kernel_sizes, padding="same", input_shape=(50, 50, 1), kernel_regularizer=regularizers.l2(0.001)))
    if(activation == 'relu'):
        model.add(Activation('relu'))
    else:
        model.add(LeakyReLU())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    model.add(Conv2D(neurons_bk2, kernel_size=kernel_sizes, border_mode="same", kernel_regularizer=regularizers.l2(0.001)))
    if(activation == 'relu'):
        model.add(Activation('relu'))
    else:
        model.add(LeakyReLU())
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    model.add(Flatten())
    model.add(Dense(500))
    if(activation == 'relu'):
        model.add(Activation('relu'))
    else:
        model.add(LeakyReLU())

    model.add(Dense(28))
    model.add(Activation("softmax"))
    
#     model.summary()
    if (optomizer == 'sgd'):
        model.compile(loss="categorical_crossentropy", optimizer=SGD(lr=learn_rate), metrics=["accuracy"])
    else:
        model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=learn_rate), metrics=["accuracy"])
    return model

In [7]:
early_stop = K.callbacks.EarlyStopping(monitor='val_acc',
                              min_delta=0,
                              patience=3,
                              verbose=0, mode='max')

In [8]:

kernel_sizes=[3, 5]
neurons_bk1=[10, 20, 30]
neurons_bk2=[50, 100]
learn_rate=[0.01, 0.005]
optomizer=['sgd', 'adam']
activation=['relu', 'leakrelu']



In [9]:
INPUT_SHAPE = (50, 50, 1)
NB_EPOCH = 20
BATCH_SIZE = 32
VERBOSE = 1
OPTIMIZER = SGD()
CLASSES = 28
STEPS = (int)(train_generator.n/BATCH_SIZE)

In [12]:
count = 0
max_f1 = 0
for ks in kernel_sizes:
    for n_bk1 in neurons_bk1:
        for n_bk2 in  neurons_bk2:
            for lr in learn_rate:
                for opt in optomizer:
                    for act in activation:
                        count += 1
                        print('{}.'.format(count))
                        model = Create_model(kernel_sizes=ks,
                                            neurons_bk1=n_bk1,
                                            neurons_bk2=n_bk2,
                                            learn_rate=lr,
                                            optomizer=opt,
                                            activation=act
                                            )
                        history = model.fit_generator(
                            train_generator,
                            steps_per_epoch=STEPS,
                            epochs=NB_EPOCH,
                            verbose=VERBOSE,
                            validation_data=validation_generator,
                    #         validation_steps=STEPS,
                            callbacks = [early_stop]
                        )
                        f1_score_val = F1ScoreCalculator(test_generator, model)
                        print ("F1-Score = {}".format(f1_score_val))
                        if (f1_score_val > max_f1):
                            best_model = model
                            max_f1 = f1_score_val
                            printer = '{}. kernel_sizes= {} | neurons_bk1= {} | neurons_bk2= {} | learn_rate={} | optomizer={} | activation={}'.format(
                                    count,
                                    ks, 
                                    n_bk1, 
                                    n_bk2, 
                                    lr,
                                    opt,
                                    act)
                            print ('Best F1-Score')
                            
                        del model
                        del history
                        K.backend.clear_session()
                        K.backend.get_session()
                
                        print ('------ END MODEL ------\n\n')

print(printer)

1.
kernel_sizes= 3 | neurons_bk1= 10 | neurons_bk2= 50 | learn_rate=0.01 | optomizer=sgd | activation=relu


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(50, kernel_size=3, kernel_regularizer=<keras.reg..., padding="same")`


Epoch 1/20
392/392 [==============================] - 6s 16ms/step - loss: 15.7548 - acc: 0.0230 - val_loss: 15.6065 - val_acc: 0.0339
Epoch 2/20
392/392 [==============================] - 6s 15ms/step - loss: 15.6184 - acc: 0.0332 - val_loss: 15.6060 - val_acc: 0.0339
Epoch 3/20
392/392 [==============================] - 6s 15ms/step - loss: 15.7823 - acc: 0.0230 - val_loss: 15.6054 - val_acc: 0.0339
Epoch 4/20
392/392 [==============================] - 6s 15ms/step - loss: 15.7407 - acc: 0.0255 - val_loss: 15.6049 - val_acc: 0.0339
F1-Score = 0.03613086770981508
Best F1-Score
------ END MODEL ------


2.
kernel_sizes= 3 | neurons_bk1= 10 | neurons_bk2= 50 | learn_rate=0.01 | optomizer=sgd | activation=leakrelu
Epoch 1/20
392/392 [==============================] - 6s 16ms/step - loss: 15.5058 - acc: 0.0408 - val_loss: 15.6199 - val_acc: 0.0339
Epoch 2/20
392/392 [==============================] - 6s 15ms/step - loss: 15.5494 - acc: 0.0383 - val_loss: 15.6191 - val_acc: 0.0339
Epoch 3/

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(100, kernel_size=3, kernel_regularizer=<keras.reg..., padding="same")`


Epoch 1/20
392/392 [==============================] - 9s 22ms/step - loss: 15.6043 - acc: 0.0332 - val_loss: 15.6294 - val_acc: 0.0316
Epoch 2/20
392/392 [==============================] - 8s 20ms/step - loss: 15.5215 - acc: 0.0383 - val_loss: 15.6291 - val_acc: 0.0316
Epoch 3/20
392/392 [==============================] - 8s 20ms/step - loss: 15.7679 - acc: 0.0230 - val_loss: 15.6288 - val_acc: 0.0316
Epoch 4/20
392/392 [==============================] - 8s 21ms/step - loss: 15.7265 - acc: 0.0255 - val_loss: 15.6285 - val_acc: 0.0316
F1-Score = 0.032716927453769556
------ END MODEL ------


10.
kernel_sizes= 3 | neurons_bk1= 10 | neurons_bk2= 100 | learn_rate=0.01 | optomizer=sgd | activation=leakrelu
Epoch 1/20
392/392 [==============================] - 9s 22ms/step - loss: 15.5157 - acc: 0.0383 - val_loss: 15.6589 - val_acc: 0.0310
Epoch 2/20
392/392 [==============================] - 8s 21ms/step - loss: 15.5825 - acc: 0.0357 - val_loss: 15.6582 - val_acc: 0.0310
Epoch 3/20
392/392 

...

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(100, kernel_size=5, kernel_regularizer=<keras.reg..., padding="same")`


Epoch 1/20
392/392 [==============================] - 9s 22ms/step - loss: 15.6164 - acc: 0.0332 - val_loss: 15.6911 - val_acc: 0.0298
Epoch 2/20
392/392 [==============================] - 8s 21ms/step - loss: 15.8013 - acc: 0.0230 - val_loss: 15.6902 - val_acc: 0.0298
Epoch 3/20
392/392 [==============================] - 8s 21ms/step - loss: 15.6771 - acc: 0.0306 - val_loss: 15.6894 - val_acc: 0.0298
Epoch 4/20
392/392 [==============================] - 8s 20ms/step - loss: 15.5530 - acc: 0.0383 - val_loss: 15.6886 - val_acc: 0.0298
F1-Score = 0.03783783783783784
------ END MODEL ------


58.
kernel_sizes= 5 | neurons_bk1= 10 | neurons_bk2= 100 | learn_rate=0.01 | optomizer=sgd | activation=leakrelu
Epoch 1/20
392/392 [==============================] - 9s 22ms/step - loss: 15.5000 - acc: 0.0434 - val_loss: 15.6072 - val_acc: 0.0368
Epoch 2/20
392/392 [==============================] - 8s 21ms/step - loss: 15.7480 - acc: 0.0281 - val_loss: 15.6059 - val_acc: 0.0368
Epoch 3/20
392/392 [

Epoch 4/20
392/392 [==============================] - 8s 19ms/step - loss: 15.8163 - acc: 0.0204 - val_loss: 15.5470 - val_acc: 0.0368
F1-Score = 0.03869132290184922
------ END MODEL ------


69.
kernel_sizes= 5 | neurons_bk1= 20 | neurons_bk2= 50 | learn_rate=0.005 | optomizer=sgd | activation=relu
Epoch 1/20
392/392 [==============================] - 6s 16ms/step - loss: 15.5743 - acc: 0.0357 - val_loss: 15.5553 - val_acc: 0.0368
Epoch 2/20
392/392 [==============================] - 6s 15ms/step - loss: 15.4089 - acc: 0.0459 - val_loss: 15.5551 - val_acc: 0.0368
Epoch 3/20
392/392 [==============================] - 6s 15ms/step - loss: 15.4909 - acc: 0.0408 - val_loss: 15.5548 - val_acc: 0.0368
Epoch 4/20
392/392 [==============================] - 6s 16ms/step - loss: 15.5729 - acc: 0.0357 - val_loss: 15.5546 - val_acc: 0.0368
F1-Score = 0.03869132290184922
------ END MODEL ------


70.
kernel_sizes= 5 | neurons_bk1= 20 | neurons_bk2= 50 | learn_rate=0.005 | optomizer=sgd | activatio

392/392 [==============================] - 12s 31ms/step - loss: 15.4405 - acc: 0.0434 - val_loss: 15.6698 - val_acc: 0.0298
Epoch 2/20
392/392 [==============================] - 11s 29ms/step - loss: 15.6947 - acc: 0.0281 - val_loss: 15.6630 - val_acc: 0.0298
Epoch 3/20
392/392 [==============================] - 11s 29ms/step - loss: 15.6062 - acc: 0.0332 - val_loss: 15.6572 - val_acc: 0.0298
Epoch 4/20
392/392 [==============================] - 11s 29ms/step - loss: 15.6831 - acc: 0.0281 - val_loss: 15.6524 - val_acc: 0.0298
F1-Score = 0.03783783783783784
------ END MODEL ------


81.
kernel_sizes= 5 | neurons_bk1= 30 | neurons_bk2= 50 | learn_rate=0.01 | optomizer=sgd | activation=relu
Epoch 1/20
392/392 [==============================] - 7s 17ms/step - loss: 15.6489 - acc: 0.0306 - val_loss: 15.6118 - val_acc: 0.0351
Epoch 2/20
392/392 [==============================] - 6s 16ms/step - loss: 15.5190 - acc: 0.0408 - val_loss: 15.6109 - val_acc: 0.0351
Epoch 3/20
392/392 [============

392/392 [==============================] - 11s 29ms/step - loss: 15.7122 - acc: 0.0306 - val_loss: 15.6723 - val_acc: 0.0322
F1-Score = 0.032432432432432434
------ END MODEL ------


92.
kernel_sizes= 5 | neurons_bk1= 30 | neurons_bk2= 100 | learn_rate=0.01 | optomizer=adam | activation=leakrelu
Epoch 1/20
392/392 [==============================] - 12s 31ms/step - loss: 15.7391 - acc: 0.0383 - val_loss: 15.7628 - val_acc: 0.0351
Epoch 2/20
392/392 [==============================] - 11s 29ms/step - loss: 15.5566 - acc: 0.0459 - val_loss: 15.7021 - val_acc: 0.0351
Epoch 3/20
392/392 [==============================] - 11s 29ms/step - loss: 15.7094 - acc: 0.0332 - val_loss: 15.6569 - val_acc: 0.0351
Epoch 4/20
392/392 [==============================] - 12s 29ms/step - loss: 15.5473 - acc: 0.0408 - val_loss: 15.6240 - val_acc: 0.0351
F1-Score = 0.034139402560455195
------ END MODEL ------


93.
kernel_sizes= 5 | neurons_bk1= 30 | neurons_bk2= 100 | learn_rate=0.005 | optomizer=sgd | activat